In [4]:
%pip install aioboto3 boto3 pandas plotly requests pillow tabulate kaleido nbformat ipython python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
from utils import queue_jobs, salad_org_id, salad_project_name, reporting_api_key, reporting_url, queue_service_url, salad_headers, salad_api_base_url, delete_all_container_groups, start_all_container_groups, stop_all_container_groups
import requests
import os
import dotenv
import re
import copy
dotenv.load_dotenv(".env", override=True)


True

In [2]:
vcpu = 2
memory = 1024 * 12

replica_count_per_group = 5

create_container_group_payload = {
  "name": "replace-this",
  "replicas": replica_count_per_group,
  "autostart_policy": False,
  "container": {
    "image": "replaceme:latest",
    "resources": {
      "cpu": vcpu,
      "memory": memory,
      "gpu_classes": []
    },
    "environment_variables": {
        "REPORTING_API_KEY": reporting_api_key,
        "REPORTING_URL": reporting_url,
        "QUEUE_URL": queue_service_url,
        "QUEUE_API_KEY": reporting_api_key
    }
  }
}


def get_gpu_classes():
    url = f"{salad_api_base_url}/organizations/{salad_org_id}/gpu-classes"
    response = requests.get(url, headers=salad_headers)
    return [gpu for gpu in response.json()["items"] if gpu["name"] != "Stable Diffusion Compatible"]


def create_container_group(name, image, gpu, env={}, dry_run=False):
    payload = copy.deepcopy(create_container_group_payload)
    payload["name"] = name
    payload["container"]["image"] = image
    payload["container"]["resources"]["gpu_classes"] = [gpu]
    payload["container"]["environment_variables"].update(env)
    url = f"{salad_api_base_url}/organizations/{salad_org_id}/projects/{salad_project_name}/containers"

    if dry_run:
        print(url)
        print(payload)
        return
    response = requests.post(url, headers=salad_headers, json=payload)
    return response.json()


gpu_classes = get_gpu_classes()
def format_gpu_name(gpu_name):
    """
    Formats the GPU name to be lowercase with only alphanumeric and hyphens.
    
    Args:
    gpu_name (str): The original GPU name.
    
    Returns:
    str: The formatted GPU name.
    """
    # Convert to lowercase
    formatted_name = gpu_name.lower()
    # Replace non-alphanumeric and non-hyphen characters with nothing
    formatted_name = re.sub(r'[^a-z0-9-]', '', formatted_name)
    
    return formatted_name

In [4]:
images = {
  "stable-fast": {
    "baked": "saladtechnologies/stable-fast-qr-code:worker0.1.0-0.4.0-baked",
    "dynamic": "saladtechnologies/stable-fast-qr-code:worker0.1.0-0.4.0"
  },
  "sdnext": {
    "baked": "saladtechnologies/sdnext:worker0.1.0-122143-128713",
    "dynamic": "saladtechnologies/sdnext:worker0.1.0-dynamic"
  },
  "a1111": {
    "baked": "saladtechnologies/a1111:worker0.1.0-122143-128713",
    "dynamic": "saladtechnologies/a1111:worker0.1.0-dynamic"
  },
  "comfy": {
    "baked": "saladtechnologies/comfyui:worker0.1.0-baked",
    "dynamic": "saladtechnologies/comfyui:worker0.1.0-dynamic"
  },
}

env = {
  "stable-fast": {
    "baked": {},
    "dynamic": {
      "CIVITAI_CONTROLNET_MODEL": "122143",
      "CIVITAI_CHECKPOINT_MODEL": "128713"
    }
  },
  "sdnext": {
    "baked": {},
    "dynamic": {
      "CIVITAI_MODEL_VERSION_IDS": "122143,128713"
    }
  },
  "a1111": {
    "baked": {},
    "dynamic": {
      "CIVITAI_MODEL_VERSION_IDS": "122143,128713"
    }
  },
  "comfy": {
    "baked": {},
    "dynamic": {
      "CIVITAI_MODEL_VERSION_IDS": "122143,128713"
    }
  },
}

queue_names = []
container_groups = []

# delete_all_container_groups()

attempt = 1

In [ ]:
def create_all_container_groups():
  global attempt
  attempt += 1
  for gpu in gpu_classes:
    for image_name, image in images.items():
      for baked in ["baked", "dynamic"]:
        for image_size in ["512", "768"]:
          env_vars = env[image_name][baked]
          name = f"{image_name}-{format_gpu_name(gpu['name'])}-{baked}-{image_size}"
          env_vars["QUEUE_NAME"] = name
          env_vars["BENCHMARK_ID"] = name
          env_vars["IMAGE_SIZE"] = image_size
          print(f"Creating container group {name}...")
          queue_names.append(name)
          create_container_group(f"{name}-{attempt}", image[baked], gpu["id"], env=env_vars)
          container_groups.append(f"{name}-{attempt}")

create_all_container_groups()
print(f"Created {len(queue_names)} container groups.")